###### Import here

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from numpy import asarray
import seaborn as sns

# Diamond Price Prediction

## Goal
The goal of this competition is the prediction of the price of diamonds based on their characteristics (weight, color, quality of cut, etc.), putting into practice all the machine learning techniques you know.

## Evaluation
The evaluation metric chosen for this competition is the RMSE (Root Mean Squared Error):

https://en.wikipedia.org/wiki/Root-mean-square_deviation

Submit results
Submissions are limited to 4 times a day.
The test set is divided into a public part (with which the public leaderboard is calculated, accessible during the competition) and another private part, with which the final positions are calculated, after the end of the competition.
Predictions will be sent in the format indicated in the sample_submission.csv file in the data section.



![diamond](img/diamond2.jpg)


## Features
```
id: only for test & sample submission files, id for prediction sample identification
price: price in USD
carat: weight of the diamond
cut: quality of the cut (Fair, Good, Very Good, Premium, Ideal)
color: diamond colour, from J (worst) to D (best)
clarity: a measurement of how clear the diamond is (I1 (worst), SI2, SI1, VS2, VS1, VVS2, VVS1, IF (best))
x: length in mm
y: width in mm
z: depth in mm
depth: total depth percentage = z / mean(x, y) = 2 * z / (x + y) (43--79)
table: width of top of diamond relative to widest point (43--95)
```

## Files
```
train.csv: training dataset 
predict.csv: dataset to predict price for 
sample_submission.csv: sample submission
```


## Submission
You may select up to 1 submission to be used to count towards your final leaderboard score. If 1 submission are not selected, they will be automatically chosen based on your best submission scores on the public leaderboard. In the event that automatic selection is not suitable, manual selection instructions will be provided in the competition rules or by official forum announcement.

Your final score may not be based on the same exact subset of data as the public leaderboard, but rather a different private data subset of your full submission — your public score is only a rough indication of what your final score is.

You should thus choose submissions that will most likely be best overall, and not necessarily on the public subset.

#### File Format
Your submission should be in CSV format. You can upload this in a zip/gz/rar/7z archive, if you prefer.

#### Number of Predictions
We expect the solution file to have 13485 prediction rows. This file should have a header row. Please see sample submission file on the data page.

## First we want to have a quick reference for creating the csv (just copy/paste)

We need: 
- a module for creating the csv
- the row_list 
- open our file with "write"

```

import csv

row_list = [["SN", "Name", "Contribution"],
             [1, "Linus Torvalds", "Linux Kernel"],
             [2, "Tim Berners-Lee", "World Wide Web"],
             [3, "Guido van Rossum", "Python Programming"]]

with open('protagonist.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(row_list)
    
```

# Now we will look at the files 

In [2]:
df = pd.read_csv("diamonds-datamad1020/train.csv")

In [3]:
df

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,0.50,Ideal,D,VS2,62.3,55.0,5.11,5.07,3.17,1845
1,1,1.54,Good,I,VS1,63.6,60.0,7.30,7.33,4.65,10164
2,2,1.32,Very Good,J,SI2,61.7,60.0,6.95,7.01,4.31,5513
3,3,1.20,Ideal,I,SI1,62.1,55.0,6.83,6.79,4.23,5174
4,4,1.73,Premium,I,SI1,61.2,60.0,7.67,7.65,4.69,10957
...,...,...,...,...,...,...,...,...,...,...,...
40450,40450,0.40,Very Good,F,SI1,62.9,58.0,4.69,4.72,2.96,687
40451,40451,0.95,Premium,H,SI1,62.9,58.0,6.26,6.21,3.92,3984
40452,40452,0.63,Ideal,F,VS2,61.2,56.0,5.56,5.51,3.39,2182
40453,40453,1.22,Good,H,VS2,63.8,55.0,6.77,6.71,4.30,7201


In [4]:
df.shape

(40455, 11)

In [5]:
dfp = pd.read_csv("diamonds-datamad1020/predict.csv")

In [6]:
dfp.shape

(13485, 10)

# The `predict.csv` is our final testing group that will produce our submission
- first we must process the `train.csv`
- find an adequeate model
- apply that model to the `predict.csv``
- submit the results of that prediction in `csv` form to kaggle
- the submission will have 13485 rows and two columns
- We shall ignore the `sample_submission.csv`
-------

# Step 1: Clean the Data

In [9]:
# Great! we are not missing any information
df.isnull().sum()

id         0
carat      0
cut        0
color      0
clarity    0
depth      0
table      0
x          0
y          0
z          0
price      0
dtype: int64

In [10]:
# We can see every column contains values for each row
# We want to check cut, color, clarity for any irregularitites.
# We expect to find: 
# 5 unique values in cut,
# 7 unique values in color
# 8 unique values in clarity
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40455 entries, 0 to 40454
Data columns (total 11 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       40455 non-null  int64  
 1   carat    40455 non-null  float64
 2   cut      40455 non-null  object 
 3   color    40455 non-null  object 
 4   clarity  40455 non-null  object 
 5   depth    40455 non-null  float64
 6   table    40455 non-null  float64
 7   x        40455 non-null  float64
 8   y        40455 non-null  float64
 9   z        40455 non-null  float64
 10  price    40455 non-null  int64  
dtypes: float64(6), int64(2), object(3)
memory usage: 3.4+ MB


In [11]:
# 5 values in cut
df["cut"].value_counts()

Ideal        16152
Premium      10321
Very Good     9040
Good          3729
Fair          1213
Name: cut, dtype: int64

In [12]:
# 7 values in color
df["color"].value_counts()

G    8469
E    7282
F    7199
H    6210
D    5098
I    4091
J    2106
Name: color, dtype: int64

In [13]:
# 8 unique values in clarity
df["clarity"].value_counts()

SI1     9758
VS2     9272
SI2     6895
VS1     6151
VVS2    3799
VVS1    2692
IF      1321
I1       567
Name: clarity, dtype: int64

In [14]:
# We can't use dummies here because they are binary.
# We need something that will reflect their order from "good to bad" 
# Here we are preparing the vlaues of our columns to be converted to ordinal values
cut = {"Fair": 0.20, 
       "Good": 0.40, 
       "Very Good": 0.60, 
       "Premium": 0.80, 
       "Ideal": 1.00}

color = {"J": 0.14, 
         "I": 0.28, 
         "H": 0.43, 
         "G": 0.57, 
         "F": 0.71, 
         "E": 0.85, 
         "D": 1.00}

clarity = {"I1": 0.125, 
           "SI2": 0.250, 
           "SI1": 0.375,
           "VS2": 0.500,
           "VS1": 0.625,
           "VVS2": 0.750,
           "VVS1": 0.875,
           "IF": 1.00} 

In [15]:
# We make a simple substitution of the values and create a new column to reflcet those values
df["cut_scale"] = df["cut"].replace(cut)
df["color_scale"]=df["color"].replace(color)
df["clarity_scale"] = df["clarity"].replace(clarity)
df = df.drop(["cut","color","clarity"], axis=1)
df

,id,carat,depth,table,x,y,z,price,cut_scale,color_scale,clarity_scale
0,0,0.50,62.3,55.0,5.11,5.07,3.17,1845,1.0,1.00,0.500
1,1,1.54,63.6,60.0,7.30,7.33,4.65,10164,0.4,0.28,0.625
2,2,1.32,61.7,60.0,6.95,7.01,4.31,5513,0.6,0.14,0.250
3,3,1.20,62.1,55.0,6.83,6.79,4.23,5174,1.0,0.28,0.375
4,4,1.73,61.2,60.0,7.67,7.65,4.69,10957,0.8,0.28,0.375
...,...,...,...,...,...,...,...,...,...,...,...
40450,40450,0.40,62.9,58.0,4.69,4.72,2.96,687,0.6,0.71,0.375
40451,40451,0.95,62.9,58.0,6.26,6.21,3.92,3984,0.8,0.43,0.375
40452,40452,0.63,61.2,56.0,5.56,5.51,3.39,2182,1.0,0.71,0.500
40453,40453,1.22,63.8,55.0,6.77,6.71,4.30,7201,0.4,0.43,0.500


In [16]:
df.to_pickle("cleaned.pkl")

We are happy with the results everything seems to be in order
<br>**End of exploration**

![clean pipes](img/cleanpipes.jpeg)